In [1]:
#importei dotenv para puxar a api a partir de um arquivo local

from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import os

from groq import Groq

#Aqui estou fazendo uma chamando a API

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
#chat completion inicia um chat que nesse caso está recebendo uma mensagem pré definida a título de exemplo.
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, which is essential for applications that require real-time or near-real-time processing, such as chatbots, virtual assistants, and language translation software.
2. **Improved User Experience**: Fast language models enable applications to respond quickly to user input, providing a seamless and interactive experience. This is particularly important for applications that require immediate feedback, such as language translation, sentiment analysis, and text summarization.
3. **Scalability**: Fast language models can handle a large volume of requests and data, making them ideal for large-scale applications, such as social media platforms, search engines, and content management systems.
4. **Cost-Effectiveness**: Fast language models can reduce the com

In [61]:
#aqui criamos a classe Agent, que recebe client e o prompt do sistema system como entrada, além de criar o self.messages, que será o histórico do nosso chat
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
#call permite que a instância da classe Agent seja usada como um função, recebendo a mensagem do usuário como entrada para gerar um resultado result
    def __call__(self, message=""):
        if message: 
            self.messages.append({"role":"user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
#função que irá ser executado no __call__, ela inicia o modelo de linguagem    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model = "llama3-70b-8192"
        )
        return completion.choices[0].message.content

In [62]:
#prompt do sistema
system_prompt = """

You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you habe been asked.
Use Action to run one of the action available to you - then return PAUSE.
Observation will be the result of running those actions.

You available actions are:

calculat:
e.g. calculate: 4 * 7 / 3
Runs a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

get_planet_relative_position:
e.g. get_planet_relative_position: Earth
returns Earths ordinal position as a string

Example session:

Question: What is the mass of Earth times 2 and what is the relative ordinal position?
Thought: I neet to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this: 

Observation: 5.972e24

Thought: I need to multiply this by 2 
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 5.972e24

Thoutht: I need to multiply this by 2 
Action: calculate  5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1.1944x10e25

Thoutht: now I need to find what is the relative position
Action: get_planet_relative_location: earth
PAUSE


If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,944x10e25 and your relative position is third. 

Now its your turn:

""".strip()

In [ ]:
# Ferramentas (Tools)
def calculate(operation):
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case _:
            return 0.0

def get_planet_relative_position(planet):
    match planet.lower():
        case "earth":
            return "third"
        case "mercury":
            return "first"
        case "venus":
            return "second"
        case "mars":
            return "fourth"
        case "jupiter":
            return "fifth"
        case "saturn":
            return "sixty"
        case _:
            return None

    

In [64]:
neil_tyson = Agent(client, system_prompt)

In [70]:
result = neil_tyson("what is the mass of mercury and what is your relative position to solar star?")
print(result)

Answer: The mass of Mercury is 3.302e23 kg and its relative position to the solar star is first.


In [71]:
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nget_planet_relative_position:\ne.g. get_planet_relative_position: Earth\nreturns Earths ordinal position as a string\n\nExample session:\n\nQuestion: What is the mass of Earth times 2 and what is the relative ordinal position?\nThought: I neet to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThou

In [18]:
result = neil_tyson()
print(result)

Action: calculate: 5.972e24 * 5 
PAUSE


In [17]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [12]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5.


In [13]:
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I neet to fin te mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2 \nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThoutht: I need to mul

In [14]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break



In [15]:
agent_loop(10, system = system_prompt, query="what is the mass of the Earth plus the mass of Mercury, and All of it times 5")

Thought: I need to find the mass of the Earth and the mass of Mercury, and then add them together before multiplying the result by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the mass of the Earth, now I need to get the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the masses of both the Earth and Mercury, now I need to add them together.
Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: I have the sum of the masses, now I need to multiply it by 5.
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the final result, which is the mass of the Earth plus the mass of Mercury, multiplied by 5.

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.15025x10e25.
